# ***DISTRIBUCIÓN DE SEGMENTOS***

### DataFrames:

In [17]:
# Manejo de datos
import os # Directorios
import pandas as pd # Manipulación df
# Gráficas
import plotly.graph_objects as go #Para obtener librería usar: pip install plotly
from plotly.subplots import make_subplots
import plotly.io as pio # Exportar gráfica

# Obtener el directorio actual de trabajo
directorio_actual = os.getcwd()

# Directorio donde se encuentran los archivos JSON (ruta relativa)
directorio_json = os.path.join(directorio_actual, '../../db/datos_json')

# Obtener la lista de archivos JSON en el directorio
archivos_json = os.listdir(directorio_json)

# Cargar los archivos JSON y crear DataFrames
for archivo in archivos_json:
    nombre_tabla = archivo.replace('datos_', '').replace('.json', '')
    ruta_json = os.path.join(directorio_json, archivo)
    globals()[f"df_{nombre_tabla}"] = pd.read_json(ruta_json)

# Obtener todos los nombres de las variables globales
nombres_variables_globales = list(globals().keys())

# Filtrar los nombres que comienzan con "df_", contienen "alfa_q" y "pachuca"
nombres_df_filtrados = [
    nombre for nombre in nombres_variables_globales 
if nombre.startswith("df_")  and "puebla" in nombre and 'df_alfa_q_puebla' not in nombre and 'financiamientos' not in nombre and 'grupos' not in nombre and 'salarios' not in nombre and 'publicacion' not in nombre and 'df_feb_2024_puebla' not in nombre and 'poblacion' not in nombre  ]

# Imprimir la lista de DataFrames filtrados
print("Lista de DataFrames filtrados:")
nombres_df_filtrados

Lista de DataFrames filtrados:


['df_alfa_agosto_2024_puebla',
 'df_alfa_febrero_2024_puebla',
 'df_alfa_julio_2024_puebla',
 'df_alfa_junio_2024_puebla',
 'df_alfa_septiembre_2024_puebla',
 'df_jul_2023_puebla',
 'df_mar_2024_puebla',
 'df_may_2024_puebla',
 'df_sep_2023_puebla']

In [21]:
df_mar_2024_puebla.columns

Index(['id', 'Q', 'Category', 'ids', 'Propiedad', 'precio', 'Metros_total',
       'Metros_construido', 'Precio_M2_Terreno', 'Precio_M2_Construido',
       'Personas_interesadas', 'Tiempo_publicacion',
       'Promedio_Interes_Mensual', 'Estado', 'Tipo', 'Estacionamientos',
       'Recamaras', 'Banos', 'Medio_banos', 'Banos_Total', 'Cantidad_de_pisos',
       'Antiguedad', 'Seguridad', 'Fraccionamiento', 'Colonia', 'CP', 'url'],
      dtype='object')

In [22]:
# Iterar sobre cada DataFrame en la lista filtrada
for nombre_df in nombres_df_filtrados:
    # Obtener el DataFrame usando globals()
    df = globals()[nombre_df]
    
    df.rename(columns={'Categoria':'categoria','Category':'categoria','Precio':'precio' }, inplace=True)
        # Asignar el DataFrame modificado de nuevo a la variable global
    globals()[nombre_df] = df 

# Imprimir confirmación
print("Columnas renombradas en los DataFrames filtrados.")


Columnas renombradas en los DataFrames filtrados.


In [24]:
# Crear una lista de DataFrames seleccionados con las columnas específicas
dataframes_list = []
for nombre_df in nombres_df_filtrados:
    # Seleccionar las columnas 'id' y 'categoria'
    segment_df = globals()[nombre_df][['categoria','precio']]
    # Añadir el DataFrame a la lista
    dataframes_list.append(segment_df)

### CONTEO ÚNICO DE CATEGORÍAS

In [25]:
rangos_precio = {
    "E1": (0, 500000),
    "E2": (500000, 750000),
    "E3": (750001, 1000000),
    "D1": (1000001, 1250000),
    "D2": (1250001, 1500000),
    "D3": (1500001, 1750000),
    "C1": (1750001, 2000000),
    "C2": (2000001, 2250000),
    "C3": (2250001, 2500000),
    "B1": (2500001, 2750000),  
    "B2": (2750001, 3000000),  
    "B3": (3000001, 3250000),  
    "A1": (3250001, 3500000),
    "A2": (3500001, 3750000),
    "A3": (3750001, 4000000),
    "S1": (4000001, 6000000),
    "S2": (6000001, 8000000),
    "S3": (8000001, 12000000),
    "L1": (12000001, 14000000),
    "L2": (14000001, 16000000),
    "L3": (16000001, 18000000),
    "L+": (18000001, 22000000),
    "ELITE": (22000001, float('inf'))
}

# Función para asignar la categoría según el precio
def asignar_categoria(precio):
    for categoria, (limite_inferior, limite_superior) in rangos_precio.items():
        if limite_inferior <= precio < limite_superior:
            return categoria
    return None  # En caso de que el precio no caiga en ningún rango (caso raro)

In [26]:
conteo_total = {}
categorias_enlistadas = ['E1', 'E2', 'E3', 'D1', 'D2', 'D3', 'C1', 'C2', 'C3', 'B1', 'B2', 'B3', 'A1', 'A2', 'A3', 'S1', 'S2', 'S3', 'L1', 'L2', 'L3', 'L+', 'ELITE']
for df in dataframes_list:
    df.loc[:, 'categoria'] = df['precio'].apply(asignar_categoria)
    conteo_actual = df['categoria'].value_counts().to_dict()
    # Agregar el conteo actual al conteo total, incluyendo categorías no presentes con conteo 0
    for categoria in categorias_enlistadas:
        conteo_total[categoria] = conteo_total.get(categoria, 0) + conteo_actual.get(categoria, 0)

conteo_total_df = pd.DataFrame(list(conteo_total.items()), columns=['categoria', 'conteo_total'])
conteo_total_df

,categoria,conteo_total
0,E1,825
1,E2,3313
2,E3,2281
3,D1,1201
4,D2,1565
5,D3,1346
6,C1,1500
7,C2,1228
8,C3,1471
9,B1,1116


In [27]:
conteo_total = {}
categorias_enlistadas = ['E1', 'E2', 'E3', 'D1', 'D2', 'D3', 'C1', 'C2', 'C3', 'B1', 'B2', 'B3', 'A1', 'A2', 'A3', 'S1', 'S2', 'S3', 'L1', 'L2', 'L3', 'L+', 'ELITE']
for df in dataframes_list:
    df.loc[:, 'categoria'] = df['precio'].apply(asignar_categoria)
    conteo_actual = df['categoria'].value_counts().to_dict()
    # Agregar el conteo actual al conteo total, incluyendo categorías no presentes con conteo 0
    for categoria in categorias_enlistadas:
        conteo_total[categoria] = conteo_total.get(categoria, 0) + conteo_actual.get(categoria, 0)

conteo_total_df = pd.DataFrame(list(conteo_total.items()), columns=['categoria', 'conteo_total'])
# Separar los números de las letras en otra columna, excepto para 'ELITE'
conteo_total_df['segmento'] = conteo_total_df['categoria'].apply(lambda x: 'ELITE' if x == 'ELITE' else x[0])
conteo_total_df['clase'] = conteo_total_df['categoria'].apply(lambda x: '1' if x == 'ELITE' else x[1:])
conteo_total_df.drop(columns=['categoria'], inplace=True)
conteo_total_df

,conteo_total,segmento,clase
0,825,E,1
1,3313,E,2
2,2281,E,3
3,1201,D,1
4,1565,D,2
5,1346,D,3
6,1500,C,1
7,1228,C,2
8,1471,C,3
9,1116,B,1


In [28]:
#segmentos_dist = conteo_total_df.groupby(['segmento', 'clase']) # Agrupar
segmentos_dist = conteo_total_df.groupby(['segmento', 'clase'])['conteo_total'].sum()
segmentos_dist

segmento  clase
A         1         959
          2        1136
          3        1183
B         1        1116
          2        1428
          3         944
C         1        1500
          2        1228
          3        1471
D         1        1201
          2        1565
          3        1346
E         1         825
          2        3313
          3        2281
ELITE     1         377
L         +         252
          1         502
          2         291
          3         159
S         1        3847
          2        1388
          3        1259
Name: conteo_total, dtype: int64

## Porcentajes totales de registros por segmento

In [29]:
# Calcular el total de registros por cada segmento, incluyendo todas sus categorías
total_por_segmento = conteo_total_df.groupby('segmento')['conteo_total'].sum() 
# Calcular el total de registros en todos los segmentos
total_registros_total = total_por_segmento.sum()
# Calcular el porcentaje que representa cada total de segmento con respecto al total de registros en todo el DataFrame
porcentaje_total_segmento = (total_por_segmento / total_registros_total) * 100
# DataFrame con los resultados
resultados = pd.DataFrame({
    'segmento': total_por_segmento.index,
    'total_por_segmento': total_por_segmento.values,
    'porcentaje_del_total': porcentaje_total_segmento.values
})

print("Total de registros: ",total_registros_total)
#porcentaje_del_total_porsegmentos
resultados

Total de registros:  29571


,segmento,total_por_segmento,porcentaje_del_total
0,A,3278,11.085185
1,B,3488,11.795340
2,C,4199,14.199723
3,D,4112,13.905516
4,E,6419,21.707078
5,ELITE,377,1.274898
6,L,1204,4.071557
7,S,6494,21.960705


In [30]:
#Obtener el total de registros por segmento y clase
total_segmento = conteo_total_df.groupby('segmento')['conteo_total'].sum()
#Calcular los porcentajes
segmentos_percent = conteo_total_df.merge(total_segmento, on='segmento', suffixes=('', '_total'))
segmentos_percent['porcentaje'] = segmentos_percent['conteo_total'] / segmentos_percent['conteo_total_total'] * 100
#Corregir el caso de 'ELITE' de clase '1' para que no salga NaN cuando es 0
segmentos_percent['porcentaje'] = segmentos_percent['porcentaje'].fillna(0)
#Seleccionar las columnas necesarias y eliminar las columnas auxiliares
segmentos_percent = segmentos_percent[['segmento', 'clase', 'porcentaje']]
segmentos_percent

,segmento,clase,porcentaje
0,E,1,12.852469
1,E,2,51.612401
2,E,3,35.535130
3,D,1,29.207198
4,D,2,38.059339
5,D,3,32.733463
6,C,1,35.722791
7,C,2,29.245058
8,C,3,35.032151
9,B,1,31.995413


Porcentaje de cada clase en relación al % que representa su segmento del total.

In [31]:
temp_dfs = [] # Almacenar los DataFrames temporales de cada segmento
# Realizamos un bucle sobre cada segmento
for segmento in resultados['segmento'].unique():
    # Filtramos las clases correspondientes al segmento actual
    clases_segmento = segmentos_percent[segmentos_percent['segmento'] == segmento]
    
    # Calculamos el total de porcentaje del segmento actual
    total_porcentaje_segmento = resultados[resultados['segmento'] == segmento]['porcentaje_del_total'].iloc[0]
    
    # Calculamos el porcentaje que representa cada clase en relación al porcentaje del segmento
    porcentaje_clase = (clases_segmento['porcentaje'] / 100) * total_porcentaje_segmento
    
    # DataFrame temporal para el segmento actual
    temp_df = pd.DataFrame({
        'segmento': [segmento] * len(clases_segmento),
        'clase': clases_segmento['clase'].values,
        'porcentaje_clase': porcentaje_clase#.round()
    })
    
    # Agrega DataFrame temporal a la lista
    temp_dfs.append(temp_df)

# Concatena todos los DataFrames temporales en un solo DataFrame
porcentajes_clase = pd.concat(temp_dfs, ignore_index=True)
porcentajes_clase

,segmento,clase,porcentaje_clase
0,A,1,3.243042
1,A,2,3.841602
2,A,3,4.000541
3,B,1,3.773968
4,B,2,4.829055
5,B,3,3.192317
6,C,1,5.072537
7,C,2,4.152717
8,C,3,4.974468
9,D,1,4.061412


In [32]:
# Agrupar por 'segmento' y 'clase', y sumar los porcentajes
porcentajes_clase_agrupado = porcentajes_clase.groupby(['segmento', 'clase']).agg({'porcentaje_clase': 'sum'}).reset_index()

# Luego aplicar pivot
porcentajes_clase_ordenado = porcentajes_clase_agrupado.pivot(index='segmento', columns='clase', values='porcentaje_clase')
porcentajes_clase_ordenado.fillna(0, inplace=True)
porcentajes_clase_ordenado

clase,+,1,2,3
segmento,,,,
A,0.000000,3.243042,3.841602,4.000541
B,0.000000,3.773968,4.829055,3.192317
C,0.000000,5.072537,4.152717,4.974468
D,0.000000,4.061412,5.292347,4.551757
E,0.000000,2.789896,11.203544,7.713638
ELITE,0.000000,1.274898,0.000000,0.000000
L,0.852186,1.697609,0.984072,0.537689
S,0.000000,13.009367,4.693788,4.257550


## Gráfica

In [33]:
segmentos = list(porcentajes_clase_ordenado.index)
segmentos_sorted = ['E','D','C','B','A','S','L','ELITE']  # Orden deseado 
clases = porcentajes_clase_ordenado.columns # Clases como etiquetas de las series de datos
clases = ['1', '2', '3']  # Clases en el orden deseado
# Asegúrate de que todos los segmentos mencionados estén en los datos con valor 0 si no existen
for segmento in segmentos_sorted:
    if segmento not in porcentajes_clase_ordenado.index:
        porcentajes_clase_ordenado.loc[segmento] = [0] * len(clases)

# Reordenar el DataFrame para seguir el orden deseado en `segmentos_sorted`
porcentajes_clase_ordenado = porcentajes_clase_ordenado.loc[segmentos_sorted]
colores = {
    '1': '#2962ff',
    '2': '#9500ff',
    '3': '#ff0059'
}
# Almacenar las barras de cada clase
barras = []
for clase in clases:
    barras.append(go.Bar(
        y=segmentos_sorted,  # Eje y: Segmentos
        x=porcentajes_clase_ordenado[clase],  # Eje x: Porcentajes de clase
        name=f'{clase}',  # Nombre de la serie de datos
        orientation='h',  # Orientación horizontal
        text=porcentajes_clase_ordenado[clase].apply(lambda x: f'{x:.2f}%'),  # Texto con los porcentajes
        textposition='auto',  # Posición automática del texto
        marker_color=colores[clase]   # Color de las barras
    ))

fig = go.Figure(data=barras)
fig.update_layout(
    yaxis=dict(
        title='Segmento', # Título del eje y        
    ),
    legend_title="Clase",
    barmode='stack',
    legend=dict(
        orientation='h',  # Orientación horizontal de la leyenda
        yanchor='bottom',  # Anclar al borde inferior
        y=1.02,  # Colocar la leyenda justo debajo del gráfico
        xanchor='right',
        x=1
    ),
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico      
    xaxis=dict(
        title='Porcentaje', # Título del eje x
        gridcolor='#dddcda', # Color de las líneas que dividen los rangos del eje Y
        gridwidth=1,
    ),  
    margin=dict(l=10, r=10, t=10, b=10)  # Ajusta los márgenes (left, right, top, bottom)
)

fig.show()

In [34]:
# Función para guardar la gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='graficas'):
    import os
    import plotly.io as pio

    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)

    # Configuración personalizada para la gráfica
    config = {
        'displaylogo': False,  # Ocultar el logo de Plotly
        'modeBarButtonsToRemove': [
            'toImage',       # Botón para guardar como imagen
            'select2d',      # Box select
            'lasso2d',       # Lasso select
            'resetScale2d',  # Reset Axes
        ]
    }

    # Guardar la gráfica como archivo HTML
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html', config=config)

# Ejemplo de uso
guardar_grafico_como_html(fig, 'g_bar_dist_segmentos', carpeta='assets/graficas')

